In [ ]:
import pandas as pd
import numpy as np
from collections import Counter
import warnings
warnings.filterwarnings('ignore')

class DrugDatasetExplorer:
    def __init__(self):
        self.datasets = {}
        self.file_names = [
            'high_confidence.csv',
            'product_adverse_effect.csv', 
            'product_label.csv',
            'product_to_rxnorm.csv',
            'vocab_meddra_adverse_effect.csv',
            'vocab_rxnorm_ingredient.csv',
            'vocab_rxnorm_ingredient_to_product.csv',
            'vocab_rxnorm_product.csv'
        ]
    
    def load_datasets(self):
        """Load all CSV files and store basic info"""
        print("="*80)
        print("LOADING AND ANALYZING DRUG DATASETS")
        print("="*80)
        
        for file_name in self.file_names:
            try:
                df = pd.read_csv(file_name)
                self.datasets[file_name] = df
                print(f"✓ Loaded {file_name}: {df.shape[0]} rows, {df.shape[1]} columns")
            except FileNotFoundError:
                print(f"✗ File not found: {file_name}")
            except Exception as e:
                print(f"✗ Error loading {file_name}: {str(e)}")
    
    def analyze_dataset_structure(self):
        """Analyze the structure of each dataset"""
        print("\n" + "="*80)
        print("DATASET STRUCTURE ANALYSIS")
        print("="*80)
        
        for file_name, df in self.datasets.items():
            print(f"\n📊 {file_name.upper()}")
            print("-" * 60)
            print(f"Shape: {df.shape}")
            print(f"Columns: {list(df.columns)}")
            print(f"Data types:\n{df.dtypes}")
            
            # Check for missing values
            missing = df.isnull().sum()
            if missing.sum() > 0:
                print(f"Missing values:\n{missing[missing > 0]}")
            else:
                print("No missing values")
            
            # Show sample data
            print("Sample data:")
            print(df.head(3).to_string())
            print()
    
    def identify_key_columns(self):
        """Identify potential key columns for joining datasets"""
        print("\n" + "="*80)
        print("KEY COLUMN IDENTIFICATION")
        print("="*80)
        
        # Common patterns for key columns
        key_patterns = ['id', 'code', 'rxnorm', 'meddra', 'product', 'ingredient', 'cui']
        
        potential_keys = {}
        
        for file_name, df in self.datasets.items():
            keys = []
            for col in df.columns:
                col_lower = col.lower()
                if any(pattern in col_lower for pattern in key_patterns):
                    unique_count = df[col].nunique()
                    total_count = len(df)
                    uniqueness_ratio = unique_count / total_count
                    keys.append({
                        'column': col,
                        'unique_values': unique_count,
                        'total_rows': total_count,
                        'uniqueness_ratio': round(uniqueness_ratio, 3),
                        'sample_values': df[col].dropna().head(5).tolist()
                    })
            
            potential_keys[file_name] = keys
            
            print(f"\n🔑 {file_name}:")
            if keys:
                for key_info in keys:
                    print(f"  - {key_info['column']}: {key_info['unique_values']} unique values "
                          f"({key_info['uniqueness_ratio']} ratio)")
                    print(f"    Sample: {key_info['sample_values']}")
            else:
                print("  No obvious key columns found")
    
    def analyze_relationships(self):
        """Analyze potential relationships between datasets"""
        print("\n" + "="*80)
        print("RELATIONSHIP ANALYSIS")
        print("="*80)
        
        # Extract all column names and look for common patterns
        all_columns = {}
        for file_name, df in self.datasets.items():
            all_columns[file_name] = df.columns.tolist()
        
        # Find common column names or patterns
        column_similarities = {}
        for file1, cols1 in all_columns.items():
            for file2, cols2 in all_columns.items():
                if file1 != file2:
                    common_cols = set(cols1) & set(cols2)
                    if common_cols:
                        key = f"{file1} ↔ {file2}"
                        column_similarities[key] = list(common_cols)
        
        print("🔗 COMMON COLUMNS BETWEEN DATASETS:")
        if column_similarities:
            for relationship, common_cols in column_similarities.items():
                print(f"  {relationship}: {common_cols}")
        else:
            print("  No exact column name matches found")
        
        # Look for potential foreign key relationships
        print("\n🔗 POTENTIAL FOREIGN KEY RELATIONSHIPS:")
        self._analyze_foreign_keys()
    
    def _analyze_foreign_keys(self):
        """Analyze potential foreign key relationships"""
        # This function looks for columns that might reference other tables
        potential_fks = []
        
        for file1, df1 in self.datasets.items():
            for col1 in df1.columns:
                col1_values = set(df1[col1].dropna().astype(str))
                
                for file2, df2 in self.datasets.items():
                    if file1 != file2:
                        for col2 in df2.columns:
                            col2_values = set(df2[col2].dropna().astype(str))
                            
                            # Check if values in col1 exist in col2
                            intersection = col1_values & col2_values
                            if len(intersection) > 0:
                                overlap_ratio = len(intersection) / len(col1_values) if len(col1_values) > 0 else 0
                                
                                if overlap_ratio > 0.1:  # At least 10% overlap
                                    potential_fks.append({
                                        'from_table': file1,
                                        'from_column': col1,
                                        'to_table': file2,
                                        'to_column': col2,
                                        'overlap_count': len(intersection),
                                        'overlap_ratio': round(overlap_ratio, 3)
                                    })
        
        # Sort by overlap ratio
        potential_fks.sort(key=lambda x: x['overlap_ratio'], reverse=True)
        
        # Show top relationships
        for fk in potential_fks[:15]:  # Show top 15
            print(f"  {fk['from_table']}.{fk['from_column']} → {fk['to_table']}.{fk['to_column']}")
            print(f"    Overlap: {fk['overlap_count']} values ({fk['overlap_ratio']} ratio)")
    
    def generate_insights(self):
        """Generate insights about the dataset structure and purpose"""
        print("\n" + "="*80)
        print("DATASET INSIGHTS AND PURPOSE")
        print("="*80)
        
        insights = {
            'high_confidence.csv': "Likely contains high-confidence drug-adverse event associations",
            'product_adverse_effect.csv': "Maps drug products to adverse effects/reactions",
            'product_label.csv': "Contains drug product labeling information",
            'product_to_rxnorm.csv': "Maps products to RxNorm standardized drug codes",
            'vocab_meddra_adverse_effect.csv': "MedDRA vocabulary for adverse effects standardization",
            'vocab_rxnorm_ingredient.csv': "RxNorm vocabulary for drug ingredients",
            'vocab_rxnorm_ingredient_to_product.csv': "Maps RxNorm ingredients to products",
            'vocab_rxnorm_product.csv': "RxNorm vocabulary for drug products"
        }
        
        for file_name in self.datasets.keys():
            print(f"\n📋 {file_name}:")
            print(f"   Purpose: {insights.get(file_name, 'Unknown purpose')}")
            
            # Analyze data characteristics
            df = self.datasets[file_name]
            
            # Look for patterns in column names
            columns = df.columns.tolist()
            print(f"   Key characteristics:")
            
            if any('rxnorm' in col.lower() for col in columns):
                print("   - Contains RxNorm standardized drug codes")
            if any('meddra' in col.lower() for col in columns):
                print("   - Contains MedDRA standardized adverse event codes")
            if any('product' in col.lower() for col in columns):
                print("   - References drug products")
            if any('ingredient' in col.lower() for col in columns):
                print("   - References drug ingredients")
            if any('adverse' in col.lower() or 'effect' in col.lower() for col in columns):
                print("   - Contains adverse event information")
            if any('label' in col.lower() for col in columns):
                print("   - Contains labeling/regulatory information")
            if any('confidence' in col.lower() for col in columns):
                print("   - Includes confidence scores or high-confidence associations")
    
    def suggest_connections(self):
        """Suggest how datasets can be connected"""
        print("\n" + "="*80)
        print("SUGGESTED DATASET CONNECTIONS")
        print("="*80)
        
        connections = [
            {
                'description': "Core Drug-Adverse Event Analysis",
                'tables': ['high_confidence.csv', 'product_adverse_effect.csv'],
                'connection': "Both contain drug-adverse event relationships"
            },
            {
                'description': "Drug Product Standardization",
                'tables': ['product_to_rxnorm.csv', 'vocab_rxnorm_product.csv'],
                'connection': "Map products to standardized RxNorm codes"
            },
            {
                'description': "Ingredient-Product Mapping",
                'tables': ['vocab_rxnorm_ingredient.csv', 'vocab_rxnorm_ingredient_to_product.csv', 'vocab_rxnorm_product.csv'],
                'connection': "Complete ingredient-to-product relationship chain"
            },
            {
                'description': "Adverse Event Standardization",
                'tables': ['product_adverse_effect.csv', 'vocab_meddra_adverse_effect.csv'],
                'connection': "Standardize adverse events using MedDRA vocabulary"
            },
            {
                'description': "Product Information Enrichment",
                'tables': ['product_label.csv', 'product_to_rxnorm.csv'],
                'connection': "Enrich product data with labeling and standardized codes"
            }
        ]
        
        for i, conn in enumerate(connections, 1):
            print(f"\n{i}. {conn['description']}:")
            print(f"   Tables: {' + '.join(conn['tables'])}")
            print(f"   Connection: {conn['connection']}")
    
    def run_complete_analysis(self):
        """Run the complete analysis pipeline"""
        self.load_datasets()
        self.analyze_dataset_structure()
        self.identify_key_columns()
        self.analyze_relationships()
        self.generate_insights()
        self.suggest_connections()
        
        print("\n" + "="*80)
        print("ANALYSIS COMPLETE")
        print("="*80)
        print("The datasets appear to form a comprehensive drug safety and standardization system.")
        print("Key components:")
        print("- Drug product identification and standardization (RxNorm)")
        print("- Adverse event standardization (MedDRA)")
        print("- Drug-adverse event associations")
        print("- Product labeling information")
        print("- Ingredient-product relationships")

# Run the analysis
if __name__ == "__main__":
    explorer = DrugDatasetExplorer()
    explorer.run_complete_analysis()

LOADING AND ANALYZING DRUG DATASETS
✓ Loaded high_confidence.csv: 663 rows, 2 columns
✓ Loaded product_adverse_effect.csv: 28126418 rows, 7 columns
✓ Loaded product_label.csv: 59515 rows, 5 columns
✓ Loaded product_to_rxnorm.csv: 94608 rows, 2 columns
✓ Loaded vocab_meddra_adverse_effect.csv: 7177 rows, 3 columns
✓ Loaded vocab_rxnorm_ingredient.csv: 2562 rows, 3 columns
✓ Loaded vocab_rxnorm_ingredient_to_product.csv: 11590 rows, 2 columns
✓ Loaded vocab_rxnorm_product.csv: 18594 rows, 3 columns

DATASET STRUCTURE ANALYSIS

📊 HIGH_CONFIDENCE.CSV
------------------------------------------------------------
Shape: (663, 2)
Columns: ['ingredient_id', 'effect_meddra_id']
Data types:
ingredient_id       int64
effect_meddra_id    int64
dtype: object
No missing values
Sample data:
   ingredient_id  effect_meddra_id
0          68149          10037844
1           6851          10024382
2          10395          10019211


📊 PRODUCT_ADVERSE_EFFECT.CSV
-------------------------------------------